# Trabalho Presencial - AV1
**Aluno**: *Gabriely Di Folco Rocha*

Link da base utilizada: https://basedosdados.org/dataset/9fa532fb-5681-4903-b99d-01dc45fd527a?table=e3016987-c3cd-4e02-8c5d-78ec1ae516ba

In [76]:
import altair as alt
import pandas as pd
import numpy as np
import nbconvert

In [77]:
df = pd.read_csv('/content/br_ibge_pnadc_ano_regiao_grupo_idade.csv')

print(df.head())

    ano  id_regiao   sexo   grupo_idade   populacao
0  2012          1  Total         Total  16452000.0
1  2012          1  Total    0 a 4 anos   1485000.0
2  2012          1  Total   5 a 13 anos   3092000.0
3  2012          1  Total    5 a 9 anos   1648000.0
4  2012          1  Total  10 a 13 anos   1444000.0


O dataframe escolhido conta com 5 variáveis: ano, id_regiao, sexo, grupo_idade e populacao. Optei por renomear a coluna id_regiao como apenas regiao e substituir seus códigos pela região que representam segundo o IBGE, como mostra o mapeamento abaixo.

In [78]:
df = df.rename(columns={'id_regiao': 'regiao'})

mapeamento_regioes = {
    3: 'sudeste',
    4: 'sul',
    5: 'centro-oeste',
    1: 'norte',
    2: 'nordeste'
}

df['regiao'] = df['regiao'].map(mapeamento_regioes)

print(df.head())

    ano regiao   sexo   grupo_idade   populacao
0  2012  norte  Total         Total  16452000.0
1  2012  norte  Total    0 a 4 anos   1485000.0
2  2012  norte  Total   5 a 13 anos   3092000.0
3  2012  norte  Total    5 a 9 anos   1648000.0
4  2012  norte  Total  10 a 13 anos   1444000.0


1.   ano: Quantitativa e discreta em valores inteiros de 2012 a 2019
2.   regiao: Categórica e nominal
3.   sexo: Categórica e nominal
4.   grupo_idade: Categórica e ordinal
5.   populacao: Quantitativa e contínua








Para simplificar a análise, inicialmente descartei os recortes de gênero e idade do DataFrame, preserevando apenas a população por ano e região, no index original.

In [101]:
# Filtrar linhas onde 'sexo' e 'grupo_idade' são "Total"
filtered_df = df[(df["sexo"] == "Total") & (df["grupo_idade"] == "Total")].copy()

# Reorganizar as colunas (se necessário)
filtered_df = filtered_df[["ano", "regiao", "populacao"]]

# Mostrar o resultado
print(filtered_df)

       ano        regiao   populacao
0     2012         norte  16452000.0
51    2013         norte  16687000.0
102   2014         norte  16925000.0
153   2015         norte  17164000.0
204   2016         norte  17392000.0
255   2017         norte  17625000.0
306   2018         norte  17871000.0
357   2019         norte  18113000.0
408   2012      nordeste  54741000.0
459   2013      nordeste  55050000.0
510   2014      nordeste  55361000.0
561   2015      nordeste  55688000.0
612   2016      nordeste  55997000.0
663   2017      nordeste  56300000.0
714   2018      nordeste  56618000.0
765   2019      nordeste  56928000.0
816   2012       sudeste  83536000.0
867   2013       sudeste  84233000.0
918   2014       sudeste  84937000.0
969   2015       sudeste  85659000.0
1020  2016       sudeste  86347000.0
1071  2017       sudeste  87014000.0
1122  2018       sudeste  87691000.0
1173  2019       sudeste  88350000.0
1224  2012           sul  28324000.0
1275  2013           sul  28551000.0
1

Aqui selecionamos apenas os recortes totais de sexo e grupo etário, para ter uma boa ideia da quantidade de pessoas em cada região

In [93]:
brasil_df = (
    filtered_df.groupby("ano", as_index=False)
    .agg({"populacao": "sum"})
    .assign(regiao="Brasil")
)

# Reorganizar as colunas e exibir o resultado
brasil_df = brasil_df[["ano", "regiao", "populacao"]]

print(brasil_df)

    ano  regiao    populacao
0  2012  Brasil  197720000.0
1  2013  Brasil  199402000.0
2  2014  Brasil  201108000.0
3  2015  Brasil  202860000.0
4  2016  Brasil  204532000.0
5  2017  Brasil  206172000.0
6  2018  Brasil  207854000.0
7  2019  Brasil  209496000.0


Com as populações nacionais e por região em mãos, busquei maneiras de expressar a distribuição. A primeira foi um gráfico de pizza:

In [128]:
df_2019 = filtered_df[df["ano"] == 2019]

chart = (
    alt.Chart(df_2019)
    .mark_arc()
    .encode(
        theta=alt.Theta(field="populacao", type="quantitative"),
        color=alt.Color(field="regiao", type="nominal", title="Região"),
        tooltip=[
            alt.Tooltip("regiao", title="Região"),
            alt.Tooltip("populacao", title="População", format=",.0f"),
        ],
    )
    .properties(
        title="População do Brasil por Região (2019)"
    )
)

chart.show()

<ipython-input-128-feaa8eaa975a>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_2019 = filtered_df[df["ano"] == 2019]


alt.Chart(...)

Testei então um histograma para a evolução da população em determinada região:

In [127]:
df_norte = filtered_df[(df['regiao'] == 'norte')]

chart = alt.Chart(df_norte).mark_bar().encode(
    x='ano:O',
    y='populacao:Q',
    tooltip=['ano', 'populacao']
).properties(
    title='Evolução da População da Região Norte (2012-2019)'
)


chart.show()


<ipython-input-127-d68716bd11e9>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_norte = filtered_df[(df['regiao'] == 'norte')]


alt.Chart(...)

Concatenei o Brasil como região no dataframe original para aprofundar a análise:

In [105]:
filtered_df = df[(df["sexo"] == "Total") & (df["grupo_idade"] == "Total")].copy()

filtered_df = filtered_df[["ano", "regiao", "populacao"]]

brasil_df = (
    filtered_df.groupby("ano", as_index=False)
    .agg({"populacao": "sum"})
    .assign(regiao="Brasil")
)

brasil_df = brasil_df[["ano", "regiao", "populacao"]]

complete_df = pd.concat([filtered_df, brasil_df], ignore_index=True)

print(complete_df)


     ano        regiao    populacao
0   2012         norte   16452000.0
1   2013         norte   16687000.0
2   2014         norte   16925000.0
3   2015         norte   17164000.0
4   2016         norte   17392000.0
5   2017         norte   17625000.0
6   2018         norte   17871000.0
7   2019         norte   18113000.0
8   2012      nordeste   54741000.0
9   2013      nordeste   55050000.0
10  2014      nordeste   55361000.0
11  2015      nordeste   55688000.0
12  2016      nordeste   55997000.0
13  2017      nordeste   56300000.0
14  2018      nordeste   56618000.0
15  2019      nordeste   56928000.0
16  2012       sudeste   83536000.0
17  2013       sudeste   84233000.0
18  2014       sudeste   84937000.0
19  2015       sudeste   85659000.0
20  2016       sudeste   86347000.0
21  2017       sudeste   87014000.0
22  2018       sudeste   87691000.0
23  2019       sudeste   88350000.0
24  2012           sul   28324000.0
25  2013           sul   28551000.0
26  2014           sul   287

Comecei ensaiando um gráfico da evolução da população brasileira.

In [115]:
chart = alt.Chart(brasil_df).mark_line().encode(
    x='ano:O',
    y='populacao:Q',
    color='regiao:N',
    tooltip=['ano', 'regiao', 'populacao']
).properties(
    title='Evolução da População Brasileira'
)

chart.show()

alt.Chart(...)

Então, me resolvi por adicionar as regiões, permitindo uma visualização mais clara do percentual que representam na população total.

In [121]:
chart = alt.Chart(complete_df).mark_line().encode(
    x='ano:O',
    y='populacao:Q',
    color='regiao:N',  # Adiciona uma linha para cada região, com cores diferentes
    tooltip=['ano', 'regiao', 'populacao']
).properties(
    title='Evolução da População Brasileira por Região'
)

chart.show()

alt.Chart(...)

Por fim, quis ordenar a população das regiões e criar uma variável que as soma progressivamente antes de representar, de forma que sua população é a diferença entre a linha que a representa e a inferior a ela.

In [125]:
# Ordenar o DataFrame por ano e população em ordem decrescente para cada ano
complete_df_sorted = complete_df.sort_values(by=['ano', 'populacao'], ascending=[True, False])

# Calcular a soma cumulativa das populações
complete_df_sorted['populacao_cumulativa'] = complete_df_sorted.groupby('ano')['populacao'].cumsum()

# Criar o gráfico de linha com a evolução progressiva da população
chart = alt.Chart(complete_df_sorted).mark_line().encode(
    x='ano:O',
    y='populacao_cumulativa:Q',
    color='regiao:N',
    tooltip=['ano', 'regiao', 'populacao_cumulativa']  # Exibir ano, região e população cumulativa
).properties(
    title='Participação regional na população'
)

chart.show()

alt.Chart(...)

Por fim, para aproveitar os dados sobre sexo e idade, construí uma pirâmide etária:

In [181]:
# Filtrando para a região 'norte'
df_etario = df[(df['regiao'] == 'norte')]

# Agrupar as faixas etárias numa escala de 10 em 10 anos"
df_etario.loc[df_etario['grupo_idade'].isin(['0 a 4 anos', '5 a 9 anos']), 'grupo_idade'] = '0 a 9 anos'
df_etario.loc[df_etario['grupo_idade'].isin(['10 a 13 anos', '14 a 15 anos', '16 a 17 anos', '18 a 19 anos']), 'grupo_idade'] = '10 a 19 anos'
df_etario.loc[df_etario['grupo_idade'].isin(['20 a 24 anos', '25 a 29 anos']), 'grupo_idade'] = '20 a 29 anos'

# Excluindo os grupos de idade indesejados
delete = ['Total', '5 a 13 anos', '14 a 17 anos', '60 anos ou mais']
df_etario = df_etario[~df_etario['grupo_idade'].isin(delete)]

# Remover a palavra "anos" da coluna 'grupo_idade'
df_etario['grupo_idade'] = df_etario['grupo_idade'].str.replace(' anos', '')

# Definindo a escala de cor para o gênero
color_scale = alt.Scale(domain=['Homens', 'Mulheres'], range=['#1f77b4', '#e377c2'])

# Criando a base do gráfico
base = alt.Chart(df_etario).properties(width=300)

# Criando a parte da visualização para as mulheres
left = base.transform_filter(
    alt.datum.sexo == 'Mulheres'
).encode(
    alt.Y('grupo_idade:N', title='', sort=['65 ou mais', '60 a 64', '50 a 59', '40 a 49',
                                                     '30 a 39', '20 a 29', '10 a 19', '0 a 9']),
    alt.X('sum(populacao):Q').title('População'),
    alt.Color('sexo:N').scale(color_scale).legend(None)
).mark_bar().properties(title='Mulheres')

# Criando a parte da visualização para os homens
right = base.transform_filter(
    alt.datum.sexo == 'Homens'
).encode(
    alt.Y('grupo_idade:N', title='Faixa Etária', sort=['65 ou mais', '60 a 64', '50 a 59', '40 a 49',
                                                     '30 a 39', '20 a 29', '10 a 19', '0 a 9']),
    alt.X('sum(populacao):Q').title('População'),
    alt.Color('sexo:N').scale(color_scale).legend(None)
).mark_bar().properties(title='Homens')

# Concatenando as visualizações em uma única
alt.concat(right, left, spacing=5)

alt.ConcatChart(...)

# Conclusão
Tive algumas dificuldades no início da atividade para acessar o dataframe, então troquei a opção algumas vezes até achar um com tamanho que me permitisse baixá-lo em .csv mas que ainda permitisse uma análise interessante. Melhorei a segmentação do gráfico de pizza, que estava exagerada pelo retorno incorreto do df em detrimento ao filtered_df, e tive a mesma percepção com o histograma. A princípio meus gráficos de linha consideravam os potos de discretização dos anos como valendo 0 no conjunto de dados, o que resultava em um gráfico de caráter triangular.
Infelizmente, para esse DataFrame não faria sentido me apegar à media, mediana e desvio padrão dos valores, pois esses conteriam pouca informação a respeito do movimento da população brasileira. Outra decepção foi a escassez de dados, que começam apenas em 2012.
Mesmo assim, estou especialmente orgulhosa do resultado final da pirâmide etária, que exprime bem um país em transição de uma base larga para um topo largo: isto é, com decaimento das taxas de natalidade e aumento da expectativa e qualidade de vida. Também gostei bastante do gráfico de linha incremental. Gostaria de ter feito um último, com as distribuições em mapa, mas isso só me ocorreu ao final da atividade.

In [126]:
# Upload ipynb
from google.colab import files
f = files.upload()
# Convert ipynb to html
import subprocess
file0 = list(f.keys())[0]
_ = subprocess.run(["pip","install","nbconvert"])
_ = subprocess.run(["jupyter","nbconvert", file0,"--to","html"])
# download the html
files.download(file0[:-5]+"html")

Saving Untitled2.ipynb to Untitled2.ipynb


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>